# Bio-optical Backscattering (BBP) Data Explorer

## This notebook processes and visualizes bio-optical data, specifically backscattering coefficients (bbp), typically derived from oceanographic sensors or satellite data.

### Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

### Load and Prepare Data
### We load the dataset and ensure the temporal and spatial columns are correctly formatted python objects.

In [ ]:
# Replace with your actual data file
file_path = 'bbp_data.csv' 

try:
    # Reading data (assuming similar structure to the Seascape datasets previously discussed)
    data = pd.read_csv(file_path)
    
    # Ensure date/time column is a datetime object
    if 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'])
    
    print(f"Loaded {len(data)} rows.")
    print(data.head())
except Exception as e:
    print(f"Error loading file: {e}")

### Data Cleaning and Pre-processing
### Handle missing values (NaNs) and filter the data for valid ranges of backscattering coefficients (e.g., bbp > 0).

In [ ]:
# Drop rows missing essential temporal or spatial information
data = data.dropna(subset=['date', 'latitude', 'longitude'])

# Filter for valid bbp values (Bio-optical coefficients must be positive)
# Adjust 'bbp_700' to the specific wavelength column in your file
bbp_col = 'bbp_700' 
if bbp_col in data.columns:
    data = data[data[bbp_col] > 0]
    print(f"Filtered data contains {len(data)} valid records.")


### BBP Vertical Profile / Time Series Visualization.
### Visualize how BBP changes over time or depth. This helps in identifying seasonal cycles or specific events.

In [ ]:
plt.figure(figsize=(12, 6))

# Plotting BBP vs Date
plt.scatter(data['date'], data[bbp_col], c=data[bbp_col], cmap='viridis', alpha=0.6, edgecolors='w')
plt.colorbar(label=f'{bbp_col} ($m^{{-1}}$)')

plt.title('Backscattering Coefficient (BBP) Over Time', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel(f'{bbp_col} [$m^{{-1}}$]', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Spatial Distribution (Heatmap)
### Visualizing the spatial distribution of BBP values using a heatmap based on Latitude and Longitude.

In [ ]:
plt.figure(figsize=(10, 8))
plt.hexbin(data['longitude'], data['latitude'], C=data[bbp_col], gridsize=30, cmap='YlGnBu')
plt.colorbar(label=f'Mean {bbp_col}')

plt.title('Spatial Distribution of BBP', fontsize=14)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.show()

### Summary Statistics
### Calculate group statistics (e.g., monthly means) to identify trends.

In [ ]:
# Create a Month-Year column for grouping
data['month_year'] = data['date'].dt.to_period('M')

monthly_summary = data.groupby('month_year')[bbp_col].agg(['mean', 'std', 'count']).reset_index()
monthly_summary['month_year'] = monthly_summary['month_year'].astype(str)

print("Monthly Summary Statistics:")
print(monthly_summary)